In [1]:
import tensorflow as tf
import keras
from keras import Model
import numpy

Using TensorFlow backend.


In [2]:
def gen_matrix():
    return numpy.array([numpy.random.randint(10, size=(1,100)), numpy.random.randint(2, size=(1,100))])

def generator(batch_size):
    while True:

        matrix = gen_matrix()
        for i in range(batch_size-1):
            matrix = numpy.concatenate([matrix, gen_matrix()], axis=1)
        yield matrix        

In [3]:
#create model
inputs = keras.layers.Input(shape=[None], dtype='int32')

padding = keras.layers.Lambda(lambda x: tf.pad(x, paddings=[[0,0],[10,10]],
                                                       constant_values=1))(inputs)

embedding = keras.layers.Embedding(input_dim=10,
                                   output_dim=14)(padding)

convolution = keras.layers.Conv1D(filters=2*14, kernel_size=21,
                                         padding='same')(embedding)

lstm_conv = keras.layers.LSTM(64, return_sequences=True, 
               dropout=0.1, recurrent_dropout=0.1)(convolution)

lstm = keras.layers.LSTM(96, return_sequences=True, 
               dropout=0.1, recurrent_dropout=0.1)(embedding)

slicing_lstm = keras.layers.Lambda(lambda x: x[:, 10:-10])(lstm)

slicing_convolution = keras.layers.Lambda(lambda x: x[:,10:-10])(lstm_conv)

concat_layer = keras.layers.Concatenate(axis=-1)([slicing_lstm, slicing_convolution])

dense1 = keras.layers.Dense(8, activation='elu')(concat_layer)

batch_layer = keras.layers.BatchNormalization()(dense1)

dense = keras.layers.Dense(1, activation='sigmoid')(batch_layer)

squeeze = keras.layers.Lambda(lambda x: tf.squeeze(
        x, axis=-1))(dense)

model1 = Model(inputs=inputs, outputs=squeeze)
#compile model
model1.compile(loss='binary_crossentropy',optimizer='adam', metrics=[])

In [4]:
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None)         0           input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 14)     140         lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 28)     8260        embedding_1[0][0]                
____________________________________________________________________________________________

In [5]:
# model.fit
model1.fit_generator(generator(batch_size=100), steps_per_epoch=10, epochs=10)

Epoch 1/10
10/10 [==============================] - 3s 250ms/step - loss: 0.6948
Epoch 2/10
10/10 [==============================] - 2s 176ms/step - loss: 0.6936
Epoch 3/10
10/10 [==============================] - 2s 179ms/step - loss: 0.6935
Epoch 4/10
10/10 [==============================] - 2s 178ms/step - loss: 0.6934
Epoch 5/10
10/10 [==============================] - 2s 177ms/step - loss: 0.6933
Epoch 6/10
10/10 [==============================] - 2s 177ms/step - loss: 0.6934
Epoch 7/10
10/10 [==============================] - 2s 175ms/step - loss: 0.6933
Epoch 8/10
10/10 [==============================] - 2s 178ms/step - loss: 0.6933
Epoch 9/10
10/10 [==============================] - 2s 176ms/step - loss: 0.6933
Epoch 10/10
10/10 [==============================] - 2s 178ms/step - loss: 0.6933


In [6]:
prediction1 = model1.predict(numpy.array([[1,3,4,8]]))
print(prediction1)

[[0.6419542  0.6400714  0.63916177 0.63838315]]


In [7]:
# saving model
model1.save('model1_functional.h5') 